In [1]:
from __future__ import division
import os
import re
import random
import time
import binascii
from bisect import bisect_right
from heapq import heappop, heappush

In [2]:
# Simliarity Hashing adapted from:
# http://mccormickml.com/2015/06/12/minhash-tutorial-with-python-code/

#=========================================================#
#                  Initial Parameters                     #
#=========================================================#

# Shingle Size determines the length of identical consecutive characters two documents must have
# in order to be considered identical. This generally works well between 30-50.

# Too small sized shingles can result in false positive matching hashes based on short strings
# like dates and email addresses.
# if shingleSize = 18
# "The quick brown fox jumps over the lazy dog on 31 August 2018"
# "[SPAM] and [ADV] should automatically be flagged as false positive on 31 August 2018"
# will produce a matching MinHash due the the matching shingle of "31 August 2018"
#
# Too long sized shigles means short documents that are similar will not be matched due to not
# having a common shingle of shingle size.
# if shingleSize=10
# "123456789RYAN123456789"
# "123456789JACK123456789"
# will not match, since there are no 10 consecutive charaters that produce a match, despite the
# similarity
#
# In general, this method of similarity clustering is good at picking out emails that follow a
# uniform formatting, especially identifying reused phrases and links. When viewing all emails,
# many false positive groupings are identified, especially due to the standard "If you are not
# the intended recepient...", and uniform organizational message signatures. However, when
# filtered to view only TRUE POSITIVE emails, this is beneficial as it is able to detect not only near-duplicates,
# has a side effect of grouping non-similiar emails from the same external organization and
# non-similiar emails that include identical links/phone numbers, indicating similarity in source.

shingleSize = 30

# Maximum possible hashedMessageShingle.
# maxShingleID = 2**32-1 = 4294967295
# https://en.wikipedia.org/wiki/4,294,967,295

# The next largest prime number above 'maxShingleID'.
nextPrime = 4294967311

# coeffA and coeffB are random coefficients, used as random seeds for the hashing.
coeffA = 740052807
coeffB = 954911726

#=========================================================#
#                  Importing Data                         #
#=========================================================#

import pandas as pd

df_raw = pd.read_csv("emails.csv", encoding = "Latin-1", usecols=["text"])
# print df_raw.head()

#=========================================================#
#               Writing .apply function                   #
#=========================================================#

def get_min_hash(row):
    
    # Extracting the document to be hashed. In this case we want Sender + Subject + Message Body
    messageBody = row["text"]
    abdc = messageBody.encode("utf8",errors="ignore")
    
    # rawHash represents the method of identical grouping
    rawHash = binascii.crc32(abdc)& 0xffffffff
    
    # For Each Doc    : "tech.gov.sg"
    # Step 1 Shingle  : set(["tech","ech.","ch.g","h.go",".gov","gov.","ov.s","v.sg"])
    # Step 2 Checksum : [2260480018L, 383729972L, 2529353918L, 448064179L, 1999796693L, 1636650756L, 3497395457L, 2315752824L]
    # Step 3 Hash     : [2813133948L, 1283942591L, 3815850831L, 2685560488L, 2528867967L, 986435944L, 310603638L, 1820640600L]
    # Step 4 Minhash  : min(Hash) = 310603638L
    
    # Shingling Documents
    messageShingles = set([messageBody[max(0, i - shingleSize):i] for i in range(shingleSize, len(messageBody) + 1)])
    minHashCode = nextPrime + 1
    minHashShingle = ""
    for shingle in messageShingles:
        shingle = shingle.encode("utf8",errors="ignore")
        # Checksum
        shingleID = (binascii.crc32(shingle)& 0xffffffff)
        # Hash
        hashCode = (coeffA * shingleID + coeffB) % nextPrime
        if hashCode < minHashCode:
            # MinHash
            minHashCode = hashCode
            # minHashShingle helps to identify which shingle the documents have in common
            minHashShingle = shingle
    minHashShingle = minHashShingle.replace('\n','').replace('\r','')
    return minHashCode,rawHash,minHashShingle

df_raw[["Minhash","rawHash","minHashShingle"]] = df_raw.apply( lambda row: pd.Series(get_min_hash(row)), axis=1)

In [4]:
df_raw.sort_values("Minhash").to_csv("emails_minHashed.csv", encoding="utf-8", sep=",",line_terminator="\n", mode='wb')

In [3]:
# Opening the CSV in EXCEL:

# Identifying Documents with similarity matches
# 1. Select cell H2, CTRL+SHIFT+DOWN to select entire row H
# 2. Conditional formatting > Create a new rule
# 3. Select a Rule Type: Use a formula to determine which cells to format
# 4. Format values where this formula is true: "=COUNTIF((H:H),H2)>1"
# 5. Format... : set desired format

# Identifying Documents with identical matches
# 6. Select cell I2, CTRL+SHIFT+DOWN to select entire row I
# 7. Conditional formatting > Create a new rule
# 8. Select a Rule Type: Use a formula to determine which cells to format
# 9. Format values where this formula is true: "=COUNTIF((I:I),I2)>1"
# 10. Format... : set desired format

# Sorting
# 11. Sort cells by value in column H to group similar emails into rows
# 12. Sort cells by value in column I to group identical emails into rows
# 13. CTRL+A to select all rows and adjust row height